# Lab: Regression models

This chapter follows closely chapter 3 and 4 of James et al. (2023) book.

# 1. Linear models

In [ ]:
pip install ISLP

In [ ]:
import numpy as np 
import pandas as pd
from matplotlib.pyplot import subplots

import statsmodels.api as sm
from statsmodels.stats.outliers_influence \
     import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
import seaborn as sns

In [ ]:
# Setup matplotlib for graphs
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# Set global parameters
%matplotlib inline
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['figure.titlesize'] = 20
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

Let us check if we had imported everything we need using the function `dir`

In [ ]:
dir()

## 1. Linear Regression

We will use the `Boston` housing dataset. It contains median house value (`medv`) for 506 neighborhoods in the Boston Metropolitan Area. The objective is to predict `medv` using 13 predictors (=covariates) and `statsmodels`, a package implementing regression methods.

Load the data and check the columns

In [ ]:
Boston = load_data("Boston")
Boston.columns

In [ ]:
# Overview of all variables
Boston.info()

We can have more information on the single variables using the function describe.

In [ ]:
#Some descriptive statistics for all the variables
Boston.describe()

Let's call a single variable `medv`. We have three simple ways to do it: 
1. Use squared brackets as if the varaible was a component of a dictionary
2. Use  dot subscripts as if the variable was a function of the data
3. Use the `loc` function (best practice)

In [ ]:
# 1. Brackets
Boston['medv']

In [ ]:
# 2. Dot
Boston.medv

In [ ]:
# 3. The loc function
Boston.loc[:,'medv']

In [ ]:
#Descriptive statistics for medv variable only
Boston['medv'].describe()

In [ ]:
Boston['lstat'].describe()

Let's fit the following linear regression model. 
$$ medv = \beta_{0} + \beta_{1}  lstat $$
Let's see a visual relationship

In [ ]:
def make_fig():
    
    # Init figure
    fig, ax = plt.subplots(1,1)
    ax.set_title('Relationship between Lstat and median value');

    # Plot scatter and best fit line
    sns.regplot(x=Boston.lstat, y=Boston.medv, ax=ax, order=1, ci=None, scatter_kws={'color':'r', 's':20})
    ax.set_xlim(-0,40); ax.set_ylim(ymin=0)
    ax.legend(['Data','Least Square Fit']);

In [ ]:
make_fig()

We are creating the matrix with an intercept column (=1) and with the value of our predictor for each observation.

In [ ]:
X = pd.DataFrame({'intercept': np.ones(Boston.shape[0]),
                  'lstat': Boston['lstat']})
X

We now extract the response associated to each row and fit the model.

In [ ]:
y = Boston['medv'] #Extract the response
model = sm.OLS(y, X) #Specifies the model 
results = model.fit() #Fit the model 

In [ ]:
summarize(results) #see the output

What can you say? 

In [ ]:
results.summary()

In [ ]:
results.params #Shows only the fitted coefficients

## 1.1 Using transformations: fit and transform

The previous model was straitghtfoward: 1 predictor variable, linear relationship, but in reality we are going to use much more difficult models (with interaction terms, quadratic terms, etc). We can use the `sklearn` package to handle to transform the data and design our matrices with method: _transform_. A transform is an object that is created with some parameters as arguments.

We rely on `MS()` a general approach developed in `ISLP` to create a transform object similar to what we would do with `sklearn`. This transform object can then be used with statistical packages such as `statsmodels`. 

We first describe this process for our previous simple regression model. Here, the transform is created by the expression `design = MS(['lstat'])`.

Then, `fit()`takes the original array and do computations on it (e.g., means, sd, scaling). Finally, `transform()` produces the model matrix. 

In [ ]:
design = MS(['lstat']) 
design = design.fit(Boston) #Checks if the variable exists. 
X = design.transform(Boston) #Constructs the model matrix with 2 columns
X

In [ ]:
design = MS(['lstat'])
X = design.fit_transform(Boston) #Combines fit and transform in one
X[:4] #Shows only the first 4 rows

Can use `get_prediction()`to obtain predictions and do inference for $\hat{y}$ for given values of $x$.

In [ ]:
new_df = pd.DataFrame({'lstat':[5, 10, 15]}) #Create a new data frame with lstat, with the values at which we want to make prediction 
newX = design.transform(new_df) #create the corresponding matrix
newX

In [ ]:
new_predictions = results.get_prediction(newX); #compute the prediction at newX
new_predictions.predicted_mean #see the results by extracting the mean 

In [ ]:
new_predictions.conf_int(alpha=0.05) #produce 95% confidence intervals

In [ ]:
new_predictions.conf_int(obs=True, alpha=0.05) #Prediction intervals: set obs=True

For instance, the 95% confidence interval associated with an `lstat` value of 5 is (29.01, 30.60), and the 95% prediction interval is (17.57, 42.04). As expected, the confidence and prediction intervals are centered around the same point (a predicted value of 29.80 for medv when lstat equals 5), but the latter are substantially wider.

### Defining functions to visualize the data 

Define a function called `abline` and has 3 arguments `ax, b, m`. `ax` is an axis object for an existing plot, `b`is the intercept and `m` the slope. Then, we add other plotting options into `ax.plot` using `*args` which allows any number of non-named arguments and `*kwargs` allows any number of named arguments to `abline`. 

In [ ]:
def abline(ax, b, m, *args, **kwargs):
    "Add a line with slope m and intercept b to ax"
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim, *args, **kwargs)

In [ ]:
ax = Boston.plot.scatter('lstat', 'medv')
abline(ax,
       results.params.iloc[0],
       results.params.iloc[1],
       'r--', # produce a red dashed line
       linewidth=3) #make it of width 3

In [ ]:
ax = subplots(figsize=(8,8))[1]
ax.scatter(results.fittedvalues, results.resid)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--')

We add a horizontal line at 0 for reference using the ax.axhline() method, indicating it should be black `(c='k')` and have a dashed linestyle `(ls='--')`.


## 1.2 Multiple linear regression

Add an extra variable: `age`

In [ ]:
X = MS(['lstat', 'age']).fit_transform(Boston) 
model1 = sm.OLS(y, X)
results1 = model1.fit()
summarize(results1)

Shortcut to include all the variables:

In [ ]:
terms = Boston.columns.drop('medv')
terms

In [ ]:
X = MS(terms).fit_transform(Boston)
model = sm.OLS(y, X)
results = model.fit()
summarize(results)

Can see that `age`has a high $p$-value, one might want to run a regression without it. 

In [ ]:
minus_age = Boston.columns.drop(['medv', 'age']) 
Xma = MS(minus_age).fit_transform(Boston)
model1 = sm.OLS(y, Xma)
summarize(model1.fit())

### Multivariate Goodness of Fit

In [ ]:
#This shows us everything available
dir(results)

Let see the $R^2$ and RSE.

In [ ]:
results.rsquared

In [ ]:
np.sqrt(results.scale)

### Interaction terms

The tuple `("lstat","age")` can be used to include an interaction term between `lstat` and ` age`. 

In [ ]:
X = MS(['lstat',
        'age',
        ('lstat', 'age')]).fit_transform(Boston)
model2 = sm.OLS(y, X)
summarize(model2.fit())

### Non-linear transformations of the predictors

Can include a polynomial functions of any predictors using `poly()`. 

In [ ]:
X = MS([poly('lstat', degree=2), 'age']).fit_transform(Boston)
model3 = sm.OLS(y, X)
results3 = model3.fit()
summarize(results3)

Can see that the the quadratic form of lstat is statistically significant, but does it really improve the model? We can measure the extent to which it does using `anova_lm()`.

In [ ]:
anova_lm(results1, results3)

The `anova_lm()` function performs a hypothesis test comparing the two models. The null hypothesis is that the quadratic term in the bigger model is not needed, and the alternative hypothesis is that the bigger model is superior. Here the F-statistic is 177.28 and the associated p-value is zero. In this case the F-statistic is the square of the t-statistic for the quadratic term in the linear model summary for `results3` --- a consequence of the fact that these nested models differ by one degree of freedom. This provides very clear evidence that the quadratic polynomial improves the linear model. Is it surprising? 

## 1.3 Qualitative predictors

Use the `Carseats`data to predict `Sales` (child car seat sales) in 400 locations. 

In [ ]:
Carseats = load_data('Carseats')
Carseats.columns

In [ ]:
Carseats.describe()

In [ ]:
Carseats['ShelveLoc']

`ShelveLoc` is a qualitative variable. It's an indicator of the quality of the shelving location (space in the store where the seat is displayed). It takes 3 values: `Bad`, `Medium`, and `Good`. `ModelSpec()` generates dummy variables automatically, what we call _one-hot encoding_. Sum of these dummies is 1, the first column is dropped to avoid collinearity (here `Bad`).  

In [ ]:
allvars = list(Carseats.columns.drop('Sales'))
y = Carseats['Sales']
final = allvars + [('Income', 'Advertising'),
                   ('Price', 'Age')]
X = MS(final).fit_transform(Carseats)
X

In [ ]:
model = sm.OLS(y, X)
summarize(model.fit())

First line: we made `allvars` a list, so that we could add the interaction terms two lines down.

Our model-matrix builder has created a `ShelveLoc[Good]` dummy variable that takes on a value of 1 if the shelving location is good, and 0 otherwise. Same for `ShelveLoc[Medium]`.

How do you interpret `ShelveLoc[Good]`? What about `ShelveLoc[Medium]`?

# 2. Classification problems

We will use the `Smarket` data. It contains percentage returns for the S&P 500 stock index over 1,250 days, from the beginning of 2001 until the end of 2005. For each date, we have the percentage returns for each of the five previous trading days,  `Lag1`  through
 `Lag5`. We have also recorded  `Volume`  (the number of shares traded on the previous day, in billions),  `Today`  (the percentage return on the date in question) and  `Direction` (whether the market was  `Up`  or  `Down`  on this date).

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)

#Specific to this part 
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
Smarket = load_data('Smarket')
Smarket

Plot the volume over time. We see that  `Volume`
is increasing over time. In other words, the average number of shares traded
daily increased from 2001 to 2005.

In [ ]:
Smarket.plot(y='Volume')

### 2.1 Logistic Regression

We will fit a logistic regression model to predict  `Direction`  using  `Lag1`  through  `Lag5`  and  `Volume`. The `sm.GLM()`  function fits *generalized linear models*, a class of models that includes logistic regression.  We could also have used 
the function `sm.Logit()` fits a logistic regression
model directly. The syntax of
`sm.GLM()` is similar to that of `sm.OLS()`, except
that we must pass in the argument `family=sm.families.Binomial()`
in order to tell `statsmodels` to run a logistic regression rather than some other
type of generalized linear model.

In [ ]:
allvars = Smarket.columns.drop(['Today', 'Direction', 'Year']) #keep all the variables excepting today, direction, year
design = MS(allvars)
X = design.fit_transform(Smarket)
y = Smarket.Direction == 'Up' #have to define wich value of y we set
glm = sm.GLM(y,
             X,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)


What can you say? 
Few tricks to access the output: `params` to access the coefficient, `pvalues` for the $p$-values.

In [ ]:
results.params

In [ ]:
results.pvalues

As we have seen earlier, we can use the `predict` method to predict the probability that the market will go up for specific values of the $x$. 

In [ ]:
probs = results.predict()
probs[:10] #looks only at the first 10 

The results obtained are on the **probability scale** but do not allow us to make predictions. To do so, we must convert these predicted probabilities into class labels `Up` or `Down`. 
First, we create an array with 1250 rows equal to `Down`. Then, we switch the value to `Up` if the predicted value is higher than $0.5$.

In [ ]:
labels = np.array(['Down']*1250)
labels[probs>0.5] = "Up"

The `confusion_table()`
function  summarizes these predictions, showing   how
many observations were correctly or incorrectly classified. This function, adapted from a similar function in the module `sklearn.metrics`,  transposes the resulting
matrix and includes row and column labels.
The `confusion_table()` function takes as first argument the
predicted labels, and second argument the true labels.

In [ ]:
confusion_table(labels, Smarket.Direction)


The diagonal elements indicate correct predictions, the off-diagonal represent incorrect predictions. 
Let's use the `np.mean()` function to compute the fraction of days for which the
prediction was correct. 

In [ ]:
(507+145)/1250, np.mean(labels == Smarket.Direction)

What can you say? Is it a good result? What could we do next?
Next, we examine how well it predicts the *leave out* data.  This
will give a more realistic error rate, because we are interested in our model’s performance not on the data that
we used to fit the model, but rather on days in the future for which
the market’s movements are unknown.


We first create a Boolean vector
corresponding to the observations from 2001 through 2004 (*training data*). We  then
use this vector to create a held out data set of observations from
2005 (*test data*).


In [ ]:
train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
Smarket_test.shape

The object `train` is a vector of 1,250 elements, corresponding
to the observations in our data set. The elements of the vector that
correspond to observations that occurred before 2005 are set to
`True`, whereas those that correspond to observations in 2005 are
set to `False`.  `train` is a
*boolean*   array, since its
elements are `True` and `False`.  Boolean arrays can be used
to obtain a subset of the rows or columns of a data frame
using the `loc` method. For instance,
the command `Smarket.loc[train]` would pick out a submatrix of the
stock market data set, corresponding only to the dates before 2005,
since those are the ones for which the elements of `train` are
`True`.  The `~` symbol can be used to negate all of the
elements of a Boolean vector. That is, `~train` is a vector
similar to `train`, except that the elements that are `True`
in `train` get swapped to `False` in `~train`, and vice versa.
Therefore, `Smarket.loc[~train]` yields a
subset of the rows of the data frame
of the stock market data containing only the observations for which
`train` is `False`.
The output above indicates that there are 252 such
observations.

We now fit a logistic regression model using only the subset of the
observations that correspond to dates before 2005. We then obtain predicted probabilities of the
stock market going up for each of the days in our test set --- that is,
for the days in 2005.

In [ ]:
X_train, X_test = X.loc[train], X.loc[~train]
y_train, y_test = y.loc[train], y.loc[~train]
glm_train = sm.GLM(y_train,
                   X_train,
                   family=sm.families.Binomial())
results = glm_train.fit()
probs = results.predict(exog=X_test)


Now, we will compare the predictions for 2005 to the movement observed in our data for the same period. First, we are storing the test and training labels.

In [ ]:
D = Smarket.Direction
L_train, L_test = D.loc[train], D.loc[~train]


In [ ]:
labels = np.array(['Down']*252)
labels[probs>0.5] = 'Up'
confusion_table(labels, L_test)

In [ ]:
np.mean(labels == L_test), np.mean(labels != L_test)

The test accuracy is about 48% while the error rate is about 52%

### 2.2 K-nearest neighbors
 We fit the classifier
using the `fit` method. New
predictions are formed using the `predict` method
of the object returned by `fit()`. One has to choose the number of neighbors to consider. We set $K=3$.

In [ ]:
knn3 = KNeighborsClassifier(n_neighbors=3)
X_train, X_test = [np.asarray(X) for X in [X_train, X_test]]
knn3.fit(X_train, L_train)
knn3_pred = knn3.predict(X_test)
confusion_table(knn3_pred, L_test)

In [ ]:
np.mean(knn3_pred == L_test)

It performs slightly better than the logisitic regression model.

KNN is a powerful classifier, let us go further with another dataset. To show it, we will use the `Caravan`  data set. This data set includes 85
predictors that measure demographic characteristics for 5,822
individuals. The response variable is  `Purchase`, which
indicates whether or not a given individual purchases a caravan
insurance policy. In this data set, only 6% of people purchased
caravan insurance.

In [ ]:
Caravan = load_data('Caravan')
Purchase = Caravan.Purchase
Purchase.value_counts()

We create a feature dataframe including all columns except `Purchase`.

In [ ]:
feature_df = Caravan.drop(columns=['Purchase'])

KNN  predicts the class of a given test
observation by identifying the observations that are nearest to it, hence
the scale of the variables matters. Any variables that are on a large
scale will have a much larger effect on the *distance* between
the observations, and hence on the KNN accuracy, than variables that
are on a small scale. Furthermore, the
importance of scale to the KNN classifier leads to another issue: if
we measured  `salary`  in Japanese yen, or if we measured
 `age`  in minutes, then we’d get quite different classification
results from what we get if these two variables are measured in
dollars and years.

How to solve such problem? A good way to deal with it is to *standardize*  the data to have all of them on a comparable scale. This is accomplished
using the `StandardScaler()`
transformation.

In [ ]:
scaler = StandardScaler(with_mean=True, #to substract the mean
                        with_std=True, #scale the column with sd=1
                        copy=True) #copy the data and not erase it

In [ ]:
scaler.fit(feature_df) #compute the parameters for the scaling and stored in scaler
X_std = scaler.transform(feature_df) #construct the standardize x


Check if it had worked

In [ ]:
feature_std = pd.DataFrame(
                 X_std,
                 columns=feature_df.columns);
feature_std.std()


Using the function `train_test_split()`  we now split the observations into a test set,
containing 1000 observations, and a training set containing the remaining
observations. The argument `random_state=0` ensures that we get
the same split each time we rerun the code.

In [ ]:
(X_train,
 X_test,
 y_train,
 y_test) = train_test_split(np.asarray(feature_std),
                            Purchase,
                            test_size=1000,
                            random_state=0)

We now fit a KNN model on the training data using $K=1$ and evaluate its performance on the test data.

In [ ]:
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1_pred = knn1.fit(X_train, y_train).predict(X_test)
np.mean(y_test != knn1_pred), np.mean(y_test != "No")


The KNN error rate is about 11%, is it a good result? Is the overall error rate the most relevant prediction performance indicator for an insurance company with a non-trivial cost of selling insurance?

#### Tuning parameters

The number of neighbors in KNN is referred to as a *tuning parameter* (or hyperparameter*).
We do not know *a priori* what value to use. It is therefore of interest
to see how the classifier performs on test data as we vary these
parameters. This can be achieved with a `for` loop to construct a *search grid*.
Here we use a for loop to look at the accuracy of our classifier in the group predicted to purchase
insurance as we vary the number of neighbors from 1 to 5:

In [ ]:
for K in range(1,6):
    knn = KNeighborsClassifier(n_neighbors=K)
    knn_pred = knn.fit(X_train, y_train).predict(X_test)
    C = confusion_table(knn_pred, y_test)
    templ = ('K={0:d}: # predicted to rent: {1:>2},' +
            '  # who did rent {2:d}, accuracy {3:.1%}')
    pred = C.loc['Yes'].sum()
    did_rent = C.loc['Yes','Yes']
    print(templ.format(
          K,
          pred,
          did_rent,
          did_rent / pred))

Does it perform better than a logit? We use `sklearn` but it fits something like the *ridge regression* version of logistic regression. We modified it by setting the argument `C` to a very large number, hence it converges to the same solution as a logit. 

In [ ]:
logit = LogisticRegression(C=1e10, solver='liblinear') #liblinear allows convergence 
logit.fit(X_train, y_train)
logit_pred = logit.predict_proba(X_test)
logit_labels = np.where(logit_pred[:,1] > .5, 'Yes', 'No')
confusion_table(logit_labels, y_test)

If we use $0.5$ as the predicted probability cut-off for the
classifier, then we have a problem: only two of the test observations
are predicted to purchase insurance.  However, we are not required to use a
cut-off of $0.5$. If we instead predict a purchase any time the
predicted probability of purchase exceeds $0.25$, we get much better
results: we predict that 29 people will purchase insurance, and we are
correct for about 31% of these people. This is almost five times
better than random guessing!

In [ ]:
logit_labels = np.where(logit_pred[:,1]>0.25, 'Yes', 'No')
confusion_table(logit_labels, y_test)

In [ ]:
9/(20+9)


In [ ]:
import sys

print(sys.version)